<a href="https://colab.research.google.com/github/MasoudAE21/HAZOP-Nodes-Prediction/blob/main/Model_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
# import torcheval
import pickle
directory = '/content/drive/MyDrive/emp/'
savepath = "/content/drive/MyDrive/paper/"
# model_savepath = "/content/drive/MyDrive/PID/Models"

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# Set the seed value
set_seed(21)

In [ ]:
NPages = 30
# data_sheet = pd.read_excel('/content/drive/MyDrive/Final Dataset3.xlsx')
# data_sheet = pd.read_excel(savepath + 'dataset_paper.xlsx')
data_sheet = pd.read_excel("/content/drive/MyDrive/paper/dataset_paper.xlsx")
data_sheet

,Unnamed: 0.1,Unnamed: 0,Name,Pages,Nodes,Unit,class-5,class-3,class-6
0,0,0,112.1 (Fire Heater),2,1,Package,A,A,A
1,1,1,112.2 (Flare),1,1,Package,A,A,A
2,2,2,112.3 (Chemical Injection),2,1,Package,A,A,A
3,3,3,112.4 (Desalter),2,1,Package,A,A,A
4,4,5,224.3.1,1,1,Package,A,A,A
...,...,...,...,...,...,...,...,...,...
254,254,293,82,30,24,Production Unit,D,C,E
255,255,298,545,25,24,Utility and Off Site,D,C,E
256,256,299,547,24,24,Transmission Pipeline and Facilities,D,C,E
257,257,300,300,23,25,Storage Tank,E,C,E


In [ ]:
data_sheet = data_sheet[data_sheet['Nodes'] < 21]
data_sheet

,Unnamed: 0.1,Unnamed: 0,Name,Pages,Nodes,Unit,class-5,class-3,class-6
0,0,0,112.1 (Fire Heater),2,1,Package,A,A,A
1,1,1,112.2 (Flare),1,1,Package,A,A,A
2,2,2,112.3 (Chemical Injection),2,1,Package,A,A,A
3,3,3,112.4 (Desalter),2,1,Package,A,A,A
4,4,5,224.3.1,1,1,Package,A,A,A
...,...,...,...,...,...,...,...,...,...
242,242,272,527,14,19,Transmission Pipeline and Facilities,D,B,D
243,243,274,103,16,20,Transmission Pipeline and Facilities,D,B,D
244,244,275,208 (AMN),23,20,Petrochemical,D,B,D
245,245,277,428 (T5),9,20,Petrochemical,D,B,D


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# load your data back to memory when you need it

with open(savepath + 'allTensorRGB30p_RSNT50V2_all.pkl', 'rb') as fi:
    mymap = pickle.load(fi)

CNN = 'RSNT50V2'
nfeatures = 2048      # 2048 for RSNT50V2, 1280 for mobile net, 2560 for EFFB7

In [ ]:
def gather_feature(dfr):
  name = dfr['Name']
  pid = mymap[str(name)]
  x0 = np.zeros([NPages, nfeatures])                                                     # 2048 for resnet, 1280 for mobile net, 2560 for EFFB7

  for i in range(NPages):
    x0[i] = pid[i].detach().numpy()

  return x0.reshape(-1)

In [ ]:
#Splitting
from sklearn.model_selection import train_test_split
ds_train, ds_test = train_test_split(data_sheet, test_size=0.2, random_state=21, stratify=data_sheet['class-6'])
ds_train, ds_val = train_test_split(ds_train, test_size=0.2, random_state=21, stratify=ds_train['class-6'])


# nlabels = 3
X_train = []
Y_train = []
for title, info in tqdm(ds_train.iterrows()):
  X_train.append(gather_feature(info))
  Y_train.append(info['class-6'])


X_test = []
Y_test = []
for title, info in tqdm(ds_test.iterrows()):
  X_test.append(gather_feature(info))
  Y_test.append(info['class-6'])



X_val = []
Y_val = []
for title, info in tqdm(ds_val.iterrows()):
  X_val.append(gather_feature(info))
  Y_val.append(info['class-6'])

unique, counts = np.unique(Y_train, return_counts=True)
nlabels = len(unique)
print(nlabels)
wh = counts[0] / counts
print(wh)
print(dict(zip(unique, counts)))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

4
[1.         1.44444444 2.24137931 3.61111111]
{'A': 65, 'B': 45, 'C': 29, 'D': 18}


In [ ]:
# Define Balancing samplers
import imblearn
sampler = imblearn.over_sampling.SMOTE(random_state=21)
X_train_balanced, Y_train_balanced = sampler.fit_resample(X_train, Y_train)

In [ ]:
print(len(X_train_balanced[0]))
print(len(X_train_balanced))


61440
260


In [ ]:
from sklearn.preprocessing import LabelEncoder
import torch

# Encode string labels to integers using LabelEncoder
label_encoder = LabelEncoder()
le = label_encoder.fit(Y_test)
print(le.classes_)

labels_train = torch.tensor(le.transform(Y_train_balanced)).to(device)
labels_test = torch.tensor(le.transform(Y_test)).to(device)
labels_val = torch.tensor(le.transform(Y_val)).to(device)


['A' 'B' 'C' 'D']


In [ ]:
X_train_balanced = np.array(X_train_balanced).reshape(-1, NPages, nfeatures)
X_test = np.array(X_test).reshape(-1, NPages, nfeatures)
X_val = np.array(X_val).reshape(-1, NPages, nfeatures)

In [ ]:
print(X_train_balanced.shape)
print(X_test.shape)

(260, 30, 2048)
(50, 30, 2048)


In [ ]:
X_val = torch.tensor(X_val, dtype= torch.float32).to(device)
X_test = torch.tensor(X_test, dtype= torch.float32).to(device)
X_train_balanced = torch.tensor(X_train_balanced, dtype= torch.float32).to(device)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Assuming page_features, additional_input, and labels are tensors
train_dataset = TensorDataset(X_train_balanced, labels_train)
test_dataset = TensorDataset(X_test, labels_test)
val_dataset = TensorDataset(X_val, labels_val)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False)


In [ ]:
# Add option of 0 for LSTM
#
class DynamicModel(nn.Module):
    def __init__(self, input_dim, num_classes, params):
        super(DynamicModel, self).__init__()

        # Unpacking the architecture parameters from GA genes
        self.lstm_layers = params[0]  # 1 or 2 or 3
        self.fc_layers = params[1]  # 0 or 1 or 2 or 3

        # Define LSTM layers dynamically
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=params[2], num_layers=params[0], batch_first=True, bidirectional=params[6])

        # Define additional fully connected layers (FC)
        fc_input_dim = params[2]      # LSTM output
        if params[6]:
            fc_input_dim *= 2
        if self.fc_layers == 0:
            self.fc = nn.Linear(fc_input_dim, num_classes)
        elif self.fc_layers == 1:
            self.fc = nn.Sequential(
                nn.Linear(fc_input_dim, params[3]),
                nn.ReLU(),
                nn.Linear(params[3], num_classes)
            )
        elif self.fc_layers == 2:
            self.fc = nn.Sequential(
                nn.Linear(fc_input_dim, params[3]),
                nn.ReLU(),
                nn.Linear(params[3], params[4]),
                nn.ReLU(),
                nn.Linear(params[4], num_classes)
            )
        elif self.fc_layers == 3:
            self.fc = nn.Sequential(
                nn.Linear(fc_input_dim, params[3]),
                nn.ReLU(),
                nn.Linear(params[3], params[4]),
                nn.ReLU(),
                nn.Linear(params[4],params[5]),
                nn.ReLU(),
                nn.Linear(params[5], num_classes)
            )
        else:
          print('Error in building fc')
          print(self.fc_layers)
          exit()

    def forward(self, page_features):
        # LSTM forward
        lstm_out, _ = self.lstm(page_features)  # Output: (batch_size, seq_len, hidden_dim)
        lstm_out_last = lstm_out[:, -1, :]  # Take the last output of the LSTM
        # Forward through fully connected layers
        output = self.fc(lstm_out_last)

        return torch.nn.functional.softmax(output, dim=1)


In [ ]:
"""class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, inputs, targets):
        loss =
        return loss.mean()"""

'class CustomLoss(nn.Module):\n    def __init__(self):\n        super(CustomLoss, self).__init__()\n\n    def forward(self, inputs, targets):\n        loss =\n        return loss.mean()'

In [ ]:
class CustomMetric(nn.Module):
    def __init__(self, nClass):
        super(CustomMetric, self).__init__()
        self.cm = torch.zeros(nClass, nClass)

    def calcCM(self, pred, target):
        # print(pred)
        # print(target)
        for p, t in zip(pred, target):
            # print(f'{p},  {t}')
            self.cm[t.int(), p.int()] += 1
        return self.cm

    def calcAccuracy(self):
        return torch.trace(self.cm) / torch.sum(self.cm)

    def calcPrecision(self, class_idx):
        return self.cm[class_idx, class_idx] / torch.sum(self.cm[:, class_idx])

    def calcRecall(self, class_idx):
        return self.cm[class_idx, class_idx] / torch.sum(self.cm[class_idx, :])

    def calcF1(self, class_idx):
        precision = self.calcPrecision(class_idx)
        recall = self.calcRecall(class_idx)
        return 2 * (precision * recall) / (precision + recall)

    def avgF1(self):
        f1_scores = []
        for i in range(self.cm.shape[0]):
            f1_scores.append(self.calcF1(i))
        return sum(f1_scores) / len(f1_scores)

    def avgPrecision(self):
        precisions = []
        for i in range(self.cm.shape[0]):
            precisions.append(self.calcPrecision(i))
        return sum(precisions) / len(precisions)

    def avgRecall(self):
        recalls = []
        for i in range(self.cm.shape[0]):
            recalls.append(self.calcRecall(i))
        return sum(recalls) / len(recalls)

# metricc = CustomMetric(6)
# print(metricc.forward(torch.randn(6), torch.randn(6)))

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs, val_loader, patience):
    counter = 0
    best = [0, None]
    early_stop = False

    for epoch in trange(num_epochs):

        model.train()
        for page_features, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(page_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        # val
        score_calc = CustomMetric(nlabels)
        model.eval()
        with torch.no_grad():
            for page_features, labels in val_loader:
                outputs = model(page_features)
                _, predicted = torch.max(outputs, 1)
                score_calc.calcCM(pred=predicted, target=labels)
            accuracy = score_calc.calcAccuracy()
            score = accuracy # score_calc.avgF1()

            if epoch % 5 == 0:
                print(f'epoch: {epoch+1}, val_accuracy: {accuracy}, F1: {score}')

            #Early Stopping
            if score > best[0]:
                best[0] = score
                best[1] = model.state_dict()
                counter = 0
            elif epoch > 5:
                counter += 1

            if counter > patience:
                early_stop = True
                print(f'epoch: {epoch+1}, val_accuracy: {accuracy}, F1: {score}, STOP!')
                return best, epoch

    return best, epoch

In [ ]:
def evaluate_model(model, dataloader):
    score_calc = CustomMetric(nlabels)
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for page_features, labels in dataloader:
            outputs = model(page_features)
            _, predicted = torch.max(outputs, 1)
            score_calc.calcCM(pred=predicted, target=labels)
        accuracy = score_calc.calcAccuracy()
        score = score_calc.avgF1()
    return accuracy, score, score_calc


In [ ]:
# temp model try
mymdl = DynamicModel(nfeatures, nlabels, params=[1, 1, 500, 500, 500, 500, True]).to(device)

criterion = nn.CrossEntropyLoss()                                           # weight=torch.tensor(wh, dtype=torch.float32).to(device)
# optimizer = torch.optim.SGD(mymdl.parameters(), lr=0.001)
optimizer = torch.optim.Adam(mymdl.parameters(), lr=0.001)
best, epoch = train_model(model=mymdl, dataloader=train_loader, criterion=criterion, optimizer=optimizer, num_epochs=100, val_loader=val_loader, patience=45)
mymdl.load_state_dict(best[1])
val_score = evaluate_model(mymdl, val_loader)
print(f'validation score: {val_score}')
# torch.save(mdl4.state_dict(), savepath + 'mdl' + '-' + '1-2-725-95-760' + '.pt')

test_score = evaluate_model(mymdl, test_loader)
print(f'test score: {test_score}')
print(test_score[2].cm)

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 1, val_accuracy: 0.42500001192092896, F1: 0.42500001192092896
epoch: 6, val_accuracy: 0.550000011920929, F1: 0.550000011920929
epoch: 11, val_accuracy: 0.4749999940395355, F1: 0.4749999940395355
epoch: 16, val_accuracy: 0.5249999761581421, F1: 0.5249999761581421
epoch: 21, val_accuracy: 0.5, F1: 0.5
epoch: 26, val_accuracy: 0.4749999940395355, F1: 0.4749999940395355
epoch: 31, val_accuracy: 0.42500001192092896, F1: 0.42500001192092896
epoch: 36, val_accuracy: 0.5249999761581421, F1: 0.5249999761581421
epoch: 41, val_accuracy: 0.5249999761581421, F1: 0.5249999761581421
epoch: 46, val_accuracy: 0.5249999761581421, F1: 0.5249999761581421
epoch: 51, val_accuracy: 0.5249999761581421, F1: 0.5249999761581421
epoch: 52, val_accuracy: 0.550000011920929, F1: 0.550000011920929, STOP!
validation score: (tensor(0.5500), tensor(0.4935), CustomMetric())
test score: (tensor(0.5400), tensor(0.4525), CustomMetric())
tensor([[14.,  5.,  1.,  0.],
        [ 2.,  9.,  3.,  1.],
        [ 0.,  1.,  3

In [ ]:
print(evaluate_model(mymdl, train_loader)[2].cm)

tensor([[59.,  2.,  4.,  0.],
        [ 2., 48.,  8.,  7.],
        [ 0.,  4., 54.,  7.],
        [ 0.,  1.,  5., 59.]])


In [ ]:
mymdl.load_state_dict(best[1])
test_score = evaluate_model(mymdl, test_loader)
print(f'test score: {test_score}')
print(test_score[2].cm)

test score: (tensor(0.5962), tensor(nan), CustomMetric())
tensor([[14.,  6.,  0.,  0.,  0.],
        [ 1., 12.,  1.,  1.,  0.],
        [ 0.,  4.,  4.,  2.,  0.],
        [ 1.,  1.,  2.,  1.,  0.],
        [ 0.,  0.,  0.,  2.,  0.]])


In [ ]:
# Function to generate initial population
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        individual = [0,0,0,0,0,0, None]
        individual[0] = int(random.choice([1, 2, 3]))                            # Number of LSTM layers
        individual[1] = int(random.choice([0, 1, 2, 3]))                         # Number of FC hidden layers
        individual[2] = int(random.randrange(5, 1001,10))                         # LSTM to FC
        individual[3] = int(random.randrange(5, 1001,10))                         # FC hidden 1
        individual[4] = int(random.randrange(5, 1001,10))                         # FC hidden 2
        individual[5] = int(random.randrange(5, 1001,10))                         # FC hidden 3
        individual[6] = random.choice([True, False])                              # Bidirectional
        population.append(individual)
    return population

# Fitness function (train and evaluate the model)
def fitness_function(individual, train_loader, val_loader, input_dim, num_classes):
    model = DynamicModel(input_dim, num_classes, params=individual).to(device)

    # Define criterion, optimizer, and train the model
    criterion = nn.CrossEntropyLoss()                                            # weight=torch.tensor(wh, dtype=torch.float32).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Here, you'd train the model and evaluate it on the validation set
    # Return validation accuracy as fitness score
    redunt = train_model(model=model, dataloader=train_loader, criterion=criterion, optimizer=optimizer, num_epochs=100, val_loader=val_loader, patience=15)
    # [best score, state dict], epoch
    model.load_state_dict(redunt[0][1])
    val_score = evaluate_model(model, val_loader)
    print(f'validation score: {val_score}, params= {individual}')

    if val_score[0] > 81:
        torch.save(model.state_dict(), savepath + CNN + '-' + str(individual[0]) + '-' + str(individual[1]) + '-' + str(individual[2]) + '-' +str(individual[3]) + '-' + str(individual[4]) + '.pt')
    return val_score[0], model

# Selection, Crossover, and Mutation functions for GA
def select_parents(population, fitness_scores):
    # Select top N individuals based on fitness
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
    numParents = max(int(0.25*len(population)), min(4, len(population)))
    return sorted_population[:numParents]  # Select top individuals to be parents

def crossover(parents):
    child = []
    for i in range(len(parents[0])):
      idx = random.randint(0, len(parents) - 1)
      child.append(parents[idx][i])
    return child

def mutate(individual):
    # Randomly mutate some hyperparameters
    if random.random() < 0.3:  # 15% chance to mutate LSTM layer count
        individual[0] = random.choice([1, 2, 3])

    if random.random() < 0.3:  # 15% chance to mutate FC layer count
        individual[1] = random.choice([1, 2, 3])

    if random.random() < 0.3:
        individual[6] = random.choice([True, False])

    # Mutate LSTM units and FC units
    while random.random() < 0.3:
      param = random.choice([2,3,4,5])
      individual[param] = int(random.randrange(5, 1001,10))

    return individual

# Main loop for Genetic Algorithm
def genetic_algorithm(train_loader, val_loader, input_dim, num_classes, pop_size=10, generations=20):
    population = initialize_population(pop_size)

    for generation in range(generations):
        print(f"Generation {generation + 1}")

        # Evaluate fitness for each individual
        gen_results = [fitness_function(ind, train_loader, val_loader, input_dim, num_classes) for ind in tqdm(population)]
        fitness_scores = [item[0][0] for item in gen_results]
        #On Generation
        best_individual = population[np.argmax(fitness_scores)]
        print(f'best= {best_individual}, fitness= {fitness_scores[np.argmax(fitness_scores)]}')


        # Select parents
        parents = select_parents(population, fitness_scores)

        # Create next generation
        new_population = []
        new_population.append(best_individual)
        for _ in range(int(pop_size / 2)):
            child = crossover(parents)
            child = mutate(child)
            new_population.append(child)

        population = new_population  # Move to next generation

    # After final generation, select best individual
    best_individual = population[np.argmax(fitness_scores)]
    return gen_results


In [ ]:
ind = genetic_algorithm(train_loader, val_loader, nfeatures, nlabels, pop_size=10, generations=5)

Generation 1


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 1, val_accuracy: 0.4285714328289032, F1: 0.4285714328289032
epoch: 6, val_accuracy: 0.4285714328289032, F1: 0.4285714328289032
epoch: 11, val_accuracy: 0.4761904776096344, F1: 0.4761904776096344
epoch: 16, val_accuracy: 0.2380952388048172, F1: 0.2380952388048172
epoch: 21, val_accuracy: 0.4523809552192688, F1: 0.4523809552192688
epoch: 26, val_accuracy: 0.2380952388048172, F1: 0.2380952388048172
epoch: 31, val_accuracy: 0.3571428656578064, F1: 0.3571428656578064
epoch: 36, val_accuracy: 0.380952388048172, F1: 0.380952388048172
epoch: 36, val_accuracy: 0.380952388048172, F1: 0.380952388048172, STOP!
validation score: (tensor(0.3810), tensor(nan)), params= [1, 3, 885, 535, 815, 365]


  0%|          | 0/100 [00:00<?, ?it/s]

...

epoch: 1, val_accuracy: 0.4761904776096344, F1: 0.4761904776096344
epoch: 6, val_accuracy: 0.4047619104385376, F1: 0.4047619104385376
epoch: 11, val_accuracy: 0.5, F1: 0.5
epoch: 16, val_accuracy: 0.4523809552192688, F1: 0.4523809552192688
epoch: 21, val_accuracy: 0.4761904776096344, F1: 0.4761904776096344
epoch: 26, val_accuracy: 0.5476190447807312, F1: 0.5476190447807312
epoch: 31, val_accuracy: 0.5714285969734192, F1: 0.5714285969734192
epoch: 36, val_accuracy: 0.5714285969734192, F1: 0.5714285969734192
epoch: 41, val_accuracy: 0.5714285969734192, F1: 0.5714285969734192
epoch: 46, val_accuracy: 0.5714285969734192, F1: 0.5714285969734192
epoch: 51, val_accuracy: 0.523809552192688, F1: 0.523809552192688
epoch: 56, val_accuracy: 0.5952380895614624, F1: 0.5952380895614624
epoch: 60, val_accuracy: 0.5476190447807312, F1: 0.5476190447807312, STOP!
validation score: (tensor(0.5476), tensor(nan)), params= [3, 3, 605, 205, 25, 205]


  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 1, val_accuracy: 0.4523809552192688, F1: 0.4523809552192688
epoch: 6, val_accuracy: 0.6190476417541504, F1: 0.6190476417541504
epoch: 11, val_accuracy: 0.5714285969734192, F1: 0.5714285969734192
epoch: 16, val_accuracy: 0.6190476417541504, F1: 0.6190476417541504
epoch: 21, val_accuracy: 0.5476190447807312, F1: 0.5476190447807312
epoch: 22, val_accuracy: 0.5, F1: 0.5, STOP!
validation score: (tensor(0.5000), tensor(nan)), params= [3, 1, 605, 205, 795, 205]


  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 1, val_accuracy: 0.380952388048172, F1: 0.380952388048172
epoch: 6, val_accuracy: 0.4523809552192688, F1: 0.4523809552192688
epoch: 11, val_accuracy: 0.523809552192688, F1: 0.523809552192688
epoch: 16, val_accuracy: 0.5714285969734192, F1: 0.5714285969734192
epoch: 21, val_accuracy: 0.5952380895614624, F1: 0.5952380895614624
epoch: 25, val_accuracy: 0.5714285969734192, F1: 0.5714285969734192, STOP!
validation score: (tensor(0.5714), tensor(nan)), params= [3, 0, 35, 25, 25, 205]
best= [3, 3, 605, 25, 25, 225], fitness= 0.5714285969734192


In [ ]:
print(ind)

[((tensor(0.5238), tensor(nan)), DynamicModel(
  (lstm): LSTM(2048, 35, num_layers=3, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=42, out_features=625, bias=True)
    (1): ReLU()
    (2): Linear(in_features=625, out_features=5, bias=True)
  )
)), ((tensor(0.5238), tensor(0.4418)), DynamicModel(
  (lstm): LSTM(2048, 75, num_layers=2, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=82, out_features=945, bias=True)
    (1): ReLU()
    (2): Linear(in_features=945, out_features=485, bias=True)
    (3): ReLU()
    (4): Linear(in_features=485, out_features=535, bias=True)
    (5): ReLU()
    (6): Linear(in_features=535, out_features=5, bias=True)
  )
)), ((tensor(0.5238), tensor(0.4502)), DynamicModel(
  (lstm): LSTM(2048, 585, num_layers=3, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=592, out_features=625, bias=True)
    (1): ReLU()
    (2): Linear(in_features=625, out_features=5, bias=True)
  )
)), ((tensor(0.5714), tensor(0.5037

In [ ]:
for i, ss in enumerate(ind):
    accuracy, score = evaluate_model(ss[1], test_loader)
    print(f"Model {i+1}: Accuracy= {accuracy}, Score= {score}")

Model 1: Accuracy= 0.48076921701431274, Score= nan
Model 2: Accuracy= 0.4615384638309479, Score= nan
Model 3: Accuracy= 0.48076921701431274, Score= nan
Model 4: Accuracy= 0.5192307829856873, Score= nan
Model 5: Accuracy= 0.5192307829856873, Score= nan
Model 6: Accuracy= 0.5769230723381042, Score= nan
Model 7: Accuracy= 0.4615384638309479, Score= nan
Model 8: Accuracy= 0.32692307233810425, Score= nan
Model 9: Accuracy= 0.5769230723381042, Score= nan
Model 10: Accuracy= 0.5384615659713745, Score= nan
Model 11: Accuracy= 0.25, Score= nan
Model 12: Accuracy= 0.5192307829856873, Score= nan
Model 13: Accuracy= 0.26923078298568726, Score= nan
Model 14: Accuracy= 0.5384615659713745, Score= 0.4088316559791565
Model 15: Accuracy= 0.5, Score= nan
Model 16: Accuracy= 0.5, Score= nan
Model 17: Accuracy= 0.557692289352417, Score= nan
Model 18: Accuracy= 0.557692289352417, Score= nan
Model 19: Accuracy= 0.5192307829856873, Score= nan
Model 20: Accuracy= 0.5384615659713745, Score= nan
Model 21: Accura

RUN BEFORE HERE

In [ ]:
'''
print(initialize_population(3))
individual = initialize_population(1)[0]
model = DynamicModel(2048, 7, 3, params=individual)
torch.save(model.state_dict(), savepath+ '-' + str(individual[0]) + '-' + str(individual[1]) + '-' + str(individual[2]) + '-' +str(individual[3]) + '-' + str(individual[4]) + '.pt')
'''